In [1]:
import pandas
import plotly.express as px
import re
import argparse
import time

In [4]:
__version__ = "V1.2(Editor) 2023-07-12"

In [2]:
# 前置参数-debug
absolute_path = False
file_path = "F:/OneDrive/Master/Project/trans/data/"
input_filename = "0002_TSS_TES.tsv"

In [ ]:
# 前置参数
parser = argparse.ArgumentParser()
parser.add_argument("--absolute_path", dest="absolute_path", required=False, action="store_true", help="use absolute file path")
parser.add_argument("--file_path", dest="file_path", required=False, type=str, default="./", help="=./,\t the path of data directory")
parser.add_argument("--input_filename", dest="input_filename", required=False, type=str, default="0002_TSS_TES.tsv", help="=\"0002_TSS_TES.tsv\",\t the output file name of 0002.py")

args = parser.parse_args()
absolute_path = args.absolute_path
file_path = args.file_path
input_filename = args.input_filename

In [ ]:
# print paraments
print('\n')
print("[Script]{}".format(__file__))
print("[Version]{}".format(__version__))
print("[Date]{}".format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time()))))
print("[Paraments]input_filename: {}".format(input_filename))
print('\n')

In [3]:
if absolute_path is False:
    input_filename = "{}{}".format(file_path, input_filename)

In [116]:
def get_bar_plot(df, disease_type, g_or_t):
    df = df
    disease_type = disease_type
    g_or_t = g_or_t

    x_type = "{}_{}".format(disease_type, g_or_t)
    y_title = "{} proportion".format(g_or_t)

    temp_df = pandas.DataFrame(index=range(0,4),columns=["5' type", "3' type", y_title])

    temp_df.at[0,"5' type"] = "ATSS"
    temp_df.at[1,"5' type"] = "ATSS"
    temp_df.at[2,"5' type"] = "Single TSS"
    temp_df.at[3,"5' type"] = "Single TSS"

    temp_df.at[0,"3' type"] = "APA"
    temp_df.at[1,"3' type"] = "Single PAS"
    temp_df.at[2,"3' type"] = "APA"
    temp_df.at[3,"3' type"] = "Single PAS"

    temp_df.at[0,"{} proportion".format(g_or_t)] = df.at[x_type, "APA/ATSS"]
    temp_df.at[1,"{} proportion".format(g_or_t)] = 1-df.at[x_type, "APA/ATSS"]
    temp_df.at[2,"{} proportion".format(g_or_t)] = df.at[x_type, "APA/TSS"]
    temp_df.at[3,"{} proportion".format(g_or_t)] = 1-df.at[x_type, "APA/TSS"]

    fig = px.bar(temp_df, x="5' type", y=y_title, color="3' type", title=disease_type)

    return fig

In [7]:
# load file
df = pandas.read_csv(input_filename, sep='\t', index_col=0)

In [11]:
# ready for analysis
df["APA/ATSS"] = df["ATSS_APA"] / (df["ATSS_APA"] + df["ATSS_PAS"])
df["APA/TSS"] = df["TSS_APA"] / (df["TSS_APA"] + df["TSS_PAS"])

In [105]:
disease_list = list(set(['_'.join(re.split(pattern='_', string=x)[0:-1]) for x in df.index.to_list()]))

In [117]:
for i in disease_list:
    for g_or_t in ["gene", "transcript"]:
        fig = get_bar_plot(df, i, g_or_t)
        fig.write_image("{}0003_{}_{}.png".format(file_path, i, g_or_t))